In [1]:
import mediapipe as mp
import cv2
import pydirectinput

In [2]:
pip install pydirectInput


Note: you may need to restart the kernel to use updated packages.


In [4]:
mp_drawing=mp.solutions.drawing_utils
mp_holistics=mp.solutions.holistic
mp_pose=mp.solutions.pose

In [13]:
import cv2
import mediapipe as mp
import pydirectinput

# Initialize MediaPipe Holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam feed
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Track previous action
prev_action = None
prev_turn = None

# Constants for Y thresholds
Y_ACCEL = 150
Y_BRAKE = 300

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip camera for natural right-hand use
    frame = cv2.flip(frame, 1)
    height, width, _ = frame.shape
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(rgb)

    action = None
    turn = None
    debug_text = "No hand detected"

    # Use only RIGHT hand
    if results.right_hand_landmarks:
        landmarks = results.right_hand_landmarks.landmark
        index_tip = landmarks[mp_holistic.HandLandmark.INDEX_FINGER_TIP]
        thumb_tip = landmarks[mp_holistic.HandLandmark.THUMB_TIP]

        x_index = int(index_tip.x * width)
        y_index = int(index_tip.y * height)

        x_thumb = int(thumb_tip.x * width)
        y_thumb = int(thumb_tip.y * height)

        # Draw circles for landmarks
        cv2.circle(frame, (x_index, y_index), 10, (0, 255, 0), -1)  # Green - index
        cv2.circle(frame, (x_thumb, y_thumb), 10, (0, 0, 255), -1)  # Red - thumb

        # Vertical steering zones (divided into 3 equal parts)
        third = width // 3
        cv2.line(frame, (third, 0), (third, height), (0, 255, 255), 2)
        cv2.line(frame, (2 * third, 0), (2 * third, height), (0, 255, 255), 2)

        # Horizontal accel/brake zones
        cv2.line(frame, (0, Y_ACCEL), (width, Y_ACCEL), (255, 0, 0), 2)
        cv2.line(frame, (0, Y_BRAKE), (width, Y_BRAKE), (0, 0, 255), 2)

        # Action based on Y-position
        if y_index < Y_ACCEL:
            action = "accelerate"
        elif y_thumb > Y_BRAKE:
            action = "brake"
        else:
            action = None

        # Turning based on X-position of index finger
        if x_index < third:
            turn = "left"
        elif x_index > 2 * third:
            turn = "right"
        else:
            turn = None

        debug_text = f"Index: ({x_index}, {y_index}) | Thumb: ({x_thumb}, {y_thumb})"

    # Apply keypresses for acceleration/braking
    if action != prev_action:
        pydirectinput.keyUp('up')
        pydirectinput.keyUp('down')
        if action == "accelerate":
            pydirectinput.keyDown('up')
        elif action == "brake":
            pydirectinput.keyDown('down')
        prev_action = action

    # Apply keypresses for left/right turning
    if turn != prev_turn:
        pydirectinput.keyUp('left')
        pydirectinput.keyUp('right')
        if turn == "left":
            pydirectinput.keyDown('left')
        elif turn == "right":
            pydirectinput.keyDown('right')
        prev_turn = turn

    # Debug info on screen
    cv2.putText(frame, debug_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 255, 200), 2)
    cv2.putText(frame, f"Action: {action}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
    cv2.putText(frame, f"Turn: {turn}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 255), 2)

    # Show the frame
    cv2.imshow("Right Hand Control", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()


FailSafeException: PyDirectInput fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pydirectinput.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [1]:
import cv2
import mediapipe as mp
import pydirectinput
import math

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Webcam feed
cap = cv2.VideoCapture(0)
cap.set(3, 640)  # Width
cap.set(4, 480)  # Height

# Track previous actions
prev_action = None
prev_turn = None

# Function to calculate distance between two points
def get_distance(p1, p2):
    return math.sqrt((p1.x - p2.x)**2 + (p1.y - p2.y)**2)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Flip camera for natural right-hand use and improve performance
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    action = None
    turn = None
    debug_text = "No hand detected"

    # Use only the first detected hand
    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        landmarks = hand_landmarks.landmark

        # --- Gesture for Acceleration/Braking ---
        # Check if hand is open (accelerate) or closed (brake)
        # We can check the distance between the thumb tip and index finger dip
        thumb_tip = landmarks[mp_hands.HandLandmark.THUMB_TIP]
        index_finger_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        
        distance = get_distance(thumb_tip, index_finger_tip)

        if distance > 0.1:  # Threshold can be adjusted
            action = "accelerate"
        else:
            action = "brake"

        # --- Gesture for Steering ---
        # Use the angle of the hand for steering
        wrist = landmarks[mp_hands.HandLandmark.WRIST]
        middle_finger_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP]

        # Calculate angle
        angle = math.degrees(math.atan2(middle_finger_mcp.y - wrist.y,
                                        middle_finger_mcp.x - wrist.x))
        
        # Normalize angle to be more intuitive
        if angle < 0:
            angle += 180

        # Steering logic based on angle
        if 45 <= angle < 80:
            turn = "left"
        elif 100 <= angle < 135:
            turn = "right"
        else:
            turn = None

        debug_text = f"Dist: {distance:.2f} | Angle: {angle:.2f}"

    # --- Apply Keypresses ---
    # Apply keypresses for acceleration/braking
    if action != prev_action:
        if prev_action == "accelerate":
            pydirectinput.keyUp('up')
        elif prev_action == "brake":
            pydirectinput.keyUp('down')

        if action == "accelerate":
            pydirectinput.keyDown('up')
        elif action == "brake":
            pydirectinput.keyDown('down')
        prev_action = action

    # Apply keypresses for left/right turning
    if turn != prev_turn:
        if prev_turn in ["left", "right"]:
            pydirectinput.keyUp(prev_turn)
            
        if turn in ["left", "right"]:
            pydirectinput.keyDown(turn)
        prev_turn = turn

    # --- Debug info on screen ---
    cv2.putText(frame, debug_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 255, 200), 2)
    cv2.putText(frame, f"Action: {action}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 200, 0), 2)
    cv2.putText(frame, f"Turn: {turn}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (200, 200, 255), 2)

    # Show the frame
    cv2.imshow("Hand Control for Racing Game", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap.release()
cv2.destroyAllWindows()
pydirectinput.keyUp('up')
pydirectinput.keyUp('down')
pydirectinput.keyUp('left')
pydirectinput.keyUp('right')

True